# Music recommender system

One of the most used machine learning algorithms is recommendation systems. A **recommender** (or recommendation) **system** (or engine) is a filtering system which aim is to predict a rating or preference a user would give to an item, eg. a film, a product, a song, etc.

Which type of recommender can we have?   

There are two main types of recommender systems: 
- Content-based filters
- Collaborative filters
  
> Content-based filters predicts what a user likes based on what that particular user has liked in the past. On the other hand, collaborative-based filters predict what a user like based on what other users, that are similar to that particular user, have liked.

### 1) Content-based filters

Recommendations done using content-based recommenders can be seen as a user-specific classification problem. This classifier learns the user's likes and dislikes from the features of the song.

The most straightforward approach is **keyword matching**.

In a few words, the idea behind is to extract meaningful keywords present in a song description a user likes, search for the keywords in other song descriptions to estimate similarities among them, and based on that, recommend those songs to the user.

*How is this performed?*

In our case, because we are working with text and words, **Term Frequency-Inverse Document Frequency (TF-IDF)** can be used for this matching process.
  
We'll go through the steps for generating a **content-based** music recommender system.

### Importing required libraries

First, we'll import all the required libraries.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List, Dict

We have already used the **TF-IDF score before**.

Likewise, we are going to use TfidfVectorizer from the Scikit-learn package again.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset

So imagine that we have the [following dataset](https://www.kaggle.com/mousehead/songlyrics/data#). 

This dataset contains name, artist, and lyrics for *57650 songs in English*. The data has been acquired from LyricsFreak through scraping.

In [4]:
songs =pd.read_csv('unique_tracks.txt',sep='<SEP>',names=['track_id', 'song_id', 'artist_name', 'release'])

<ipython-input-4-c457e5797241>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  songs =pd.read_csv('unique_tracks.txt',sep='<SEP>',names=['track_id', 'song_id', 'artist_name', 'release'])


In [5]:
songs.drop(columns=['track_id','song_id'], inplace=True)

In [6]:
songs.head()

,artist_name,release
0,Faster Pussy cat,Silent Night
1,Karkkiautomaatti,Tanssi vaan
2,Hudson Mohawke,No One Could Ever
3,Yerba Brava,Si Vos Querés
4,Der Mystic,Tangle Of Aspens


In [33]:
df=songs.iloc[0:10000,:]

In [34]:
df.shape

(10000, 2)

After that, we use TF-IDF vectorizerthat calculates the TF-IDF score for each release , word-by-word. 

Here, we pay particular attention to the arguments we can specify.

In [35]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [36]:
lyrics_matrix = tfidf.fit_transform(df['release'].values.astype('U'))

*How do we use this matrix for a recommendation?* 

We now need to calculate the similarity of one lyric to another. We are going to use **cosine similarity**.

We want to calculate the cosine similarity of each item with every other item in the dataset. So we just pass the lyrics_matrix as argument.

In [37]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

Once we get the similarities, we'll store in a dictionary the names of the 50  most similar songs for each song in our dataset.

In [38]:
similarities = {}

In [42]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[df['release'].iloc[i]] = [(cosine_similarities[i][x], songs['release'][x], songs['artist_name'][x]) for x in similar_indices][1:]

After that, all the magic happens. We can use that similarity scores to access the most similar items and give a recommendation.

For that, we'll define our Content based recommender class.

In [58]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, release, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {release} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        release = recommendation['release']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[release][:number_songs]
        # print each item
        self._print_message(release=release, recom_song=recom_song)

Now, instantiate class

In [59]:
recommedations = ContentBasedRecommender(similarities)

Then, we are ready to pick a song from the dataset and make a recommendation.

In [60]:
recommendation = {
    "release": df['release'].iloc[10],
    "number_songs": 4 
}

In [61]:
recommedations.recommend(recommendation)

The 4 recommended songs for L'antarctique are:
Number 1:
Little Drummer Boy by Nat Stuckey with 0.0 similarity score
--------------------
Number 2:
Mama_ Talk To Your Daughter - Live by Hound Dog Taylor and The Houserockers with 0.0 similarity score
--------------------
Number 3:
Ah Yeah! by KRS-One with 0.0 similarity score
--------------------
Number 4:
Andy McGann's / George White's Favorite by Joe Derrane_ Frankie Gavin_ Brian McGrath with 0.0 similarity score
--------------------


And we can pick another random song and recommend again:

In [62]:
recommendation2 = {
    "release": songs['release'].iloc[120],
    "number_songs": 4 
}

In [63]:
recommedations.recommend(recommendation2)

The 4 recommended songs for Hot (Album Version) are:
Number 1:
Do What You Can (Album Version) by James Jackson Toth with 0.615 similarity score
--------------------
Number 2:
C.e.b. (Album Version) by No Innocent Victim with 0.615 similarity score
--------------------
Number 3:
I'm Done (album version) by Tweet with 0.615 similarity score
--------------------
Number 4:
Sometimes (Album Version) by Flamin' Groovies with 0.615 similarity score
--------------------
